In [5]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Union
import glob
import os 

In [3]:
# dataset git repo
if not os.path.exists("hate-speech-dataset"):
  !git clone https://github.com/Vicomtech/hate-speech-dataset.git
    
!python -m textblob.download_corpora

Cloning into 'hate-speech-dataset'...
remote: Enumerating objects: 10785, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 10785 (delta 6), reused 0 (delta 0), pack-reused 10771
Receiving objects: 100% (10785/10785), 1.16 MiB | 1.97 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Updating files: 100% (13339/13339), done.
/Users/arasdirekoglu/opt/anaconda3/envs/dscience/bin/python: Error while finding module specification for 'textblob.download_corpora' (ModuleNotFoundError: No module named 'textblob')


In [6]:
def readData(paths: List[str], group: str):
    pairs = []
    for p in paths:
        with open(p) as f:
            file_id = p.split('/')[-1].split('.')[0]
            pairs.append((file_id, f.read(), group))        
    return pd.DataFrame(pairs, columns=["file_id", "text", "gSet"])

In [7]:
idLabels = pd.read_csv('hate-speech-dataset/annotations_metadata.csv')[["file_id", "label"]]

In [9]:
trainDf = readData(glob.glob('./hate-speech-dataset/sampled_train/*.txt'), 'train')
testDf = readData(glob.glob('./hate-speech-dataset/sampled_test/*.txt'), 'test')

trainDf = trainDf.join(idLabels.set_index('file_id'), on='file_id')
testDf = testDf.join(idLabels.set_index('file_id'), on='file_id')

combDf = trainDf.append(testDf).reset_index(drop=True)

# print label distributions
print("Train dataset:")
print(trainDf['label'].value_counts(), "\n")
print("Test dataset:")
print(testDf['label'].value_counts(), "\n")

print("Total datase shape:")
print(combDf.shape)
combDf.head()

Train dataset:
noHate    957
hate      957
Name: label, dtype: int64 

Test dataset:
noHate    239
hate      239
Name: label, dtype: int64 

Total datase shape:
(2392, 4)


/var/folders/bs/ht_t1wp94tl3wt5xhchcxq2m0000gn/T/ipykernel_10441/450055070.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combDf = trainDf.append(testDf).reset_index(drop=True)


,file_id,text,gSet,label
0,14651837_4,Ready .... GO !!!,train,noHate
1,31713873_1,If i sent you some pic wound use be able to pu...,train,noHate
2,30586609_1,ALWAYS SPEAK OUT WHEN IN A CLASS WITH A BRAINW...,train,noHate
3,14673212_2,They fit the same DNA as the DNA found in anci...,train,noHate
4,33222250_1,I would not be surprised if this woman was spe...,train,hate


In [10]:
df = combDf[['text', 'label']]

In [11]:
df.head()

,text,label
0,Ready .... GO !!!,noHate
1,If i sent you some pic wound use be able to pu...,noHate
2,ALWAYS SPEAK OUT WHEN IN A CLASS WITH A BRAINW...,noHate
3,They fit the same DNA as the DNA found in anci...,noHate
4,I would not be surprised if this woman was spe...,hate


In [13]:
df.to_csv('whitesupremacist_hatespeech.csv', index=False)